In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
%cd C:\Users\asche\deep_learning\exam-project\kunskapskontroll_2

C:\Users\asche\deep_learning\exam-project\kunskapskontroll_2


In [4]:
from flask import Flask, render_template, request, Response, jsonify
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import base64
import sys
import logging as log
import datetime as dt
from time import sleep, time

In [5]:
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
emotion_model = load_model('final_model.h5', compile=False)

### Face and emotion detection app which opens a window with the local webcam.
Draws a rectangle on detected faces. Writes an emotion text = the predicted emotion.

In [6]:
cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
log.basicConfig(filename='webcam.log', level=log.INFO)

video_capture = cv2.VideoCapture(0)
anterior = 0

# Set the desired detection interval (in seconds)
detection_interval = 0.5

last_detection_time = time()
print("Video capture started. Select video and press 'q' to stop")

while True:
    if not video_capture.isOpened():
        print('Unable to load camera.')
        sleep(5)
        continue

    current_time = time()
    elapsed_time = current_time - last_detection_time

    if elapsed_time < detection_interval:
        # Wait for the desired interval
        sleep(detection_interval - elapsed_time)
        continue

    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    print('Number of faces detected:', len(faces))

    # Process only the first detected face (if any)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]  # Use the first detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        face_region = gray[y:y + h, x:x + w]

        # Convert grayscale image to three channels (RGB)
        face_image = cv2.cvtColor(face_region, cv2.COLOR_GRAY2RGB)
        _, buffer = cv2.imencode('.jpg', face_image)
        face_image_encoded = base64.b64encode(buffer).decode('utf-8')

        preprocessed_face = cv2.resize(face_image, (224, 224))
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)  # Add batch dimension
        preprocessed_face = np.expand_dims(preprocessed_face, axis=-1)  # Add channel dimension

        # Predict the emotion
        try:
            predicted_emotion = emotion_labels[np.argmax(emotion_model.predict(preprocessed_face))]
        except ValueError as e:
            print(f"Error predicting emotion: {e}")
            continue  # Skip this frame if prediction fails

        cv2.putText(frame, f'Emotion: {predicted_emotion}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    if anterior != len(faces):
        anterior = len(faces)
        log.info("faces: " + str(len(faces)) + " at " + str(dt.datetime.now()))

    cv2.imshow('Video', frame)

    last_detection_time = time()  # Update the last detection time

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()




Video capture started. Select video and press 'q' to stop
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 1
1/1 [==============================] - 0s 222ms/step
Number of faces detected: 1
1/1 [==============================] - 0s 78ms/step
Number of faces detected: 1
1/1 [==============================] - 0s 79ms/step
Number of faces detected: 1
1/1 [==============================] - 0s 78ms/step
Number of faces detected: 1
1/1 [==============================] - 0s 79ms/step
Number of faces detected: 1
1/1 [==============================] - 0s 80ms/step
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0
Number of faces detected: 0


In [7]:
'''

from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

def detect_faces():
    while True:
        ret, frame = video_capture.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        for (x, y, w, h) in faces:
            # Extract the face region
            face = frame[y:y+h, x:x+w]

            # Save the face region as an image
            face_image = cv2.resize(face, (224, 224))
            _, buffer = cv2.imencode('.jpg', face_image)
            face_image_encoded = base64.b64encode(buffer).decode('utf-8')

            # Preprocess the face for emotion prediction
            preprocessed_face = cv2.resize(face_image, (224, 224))
            preprocessed_face = np.expand_dims(preprocessed_face, axis=0)

            # Predict the emotion
            predicted_emotion = emotion_labels[np.argmax(emotion_model.predict(preprocessed_face))]

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Display the predicted emotion
            cv2.putText(frame, predicted_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

        _, jpeg = cv2.imencode('.jpg', frame)
        frame_bytes = jpeg.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(detect_faces(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)


'''

'\n\nfrom flask import Flask, render_template, Response\nimport cv2\n\napp = Flask(__name__)\n\nface_cascade = cv2.CascadeClassifier(\'haarcascade_frontalface_default.xml\')\nvideo_capture = cv2.VideoCapture(0)\n\ndef detect_faces():\n    while True:\n        ret, frame = video_capture.read()\n\n        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n        faces = face_cascade.detectMultiScale(\n            gray,\n            scaleFactor=1.1,\n            minNeighbors=5,\n            minSize=(30, 30)\n        )\n\n        for (x, y, w, h) in faces:\n            # Extract the face region\n            face = frame[y:y+h, x:x+w]\n\n            # Save the face region as an image\n            face_image = cv2.resize(face, (224, 224))\n            _, buffer = cv2.imencode(\'.jpg\', face_image)\n            face_image_encoded = base64.b64encode(buffer).decode(\'utf-8\')\n\n            # Preprocess the face for emotion prediction\n            preprocessed_face = cv2.resize(face_image, (224